# Team Project - Kickstarter Project

* Author: Julia Hammerer, Vanessa Mai
* Last Update: 15.06.2018



## Project Description
In this project we will look at a list of Crowdfunding Projects pulled from the Kickstarter website in 2018. Analysis will be mainly Data Explorations on the Project and may include (but not exclusively)
* compare successful and failed projects per country, and category
* look at size/funding amount of projects
* what's the value of successful projects, are there differences to failed ones?
* time series analysis 
* how much do people donate to projects in average

Here we will import all packages required for our analysis

In [ ]:
import pandas as pd
from datetime import datetime

Then first of all we import our csv file as a pandas dataframe


In [ ]:
ks_raw=pd.read_csv("ks-projects-201801.csv")

In [ ]:
ks_raw

When looking at the data we see that there are several fields with amounts (=pledged). The fields "pledged" and "goal" are in the original currency. There is also "usd pledged", usd_pledged_real", "uds_goal_real". "usd pledged" is the amount converted to us-dollar by Kickstarter. According to the description from Kaggle "usd_pledged_real" and "usd_goal_real" are converted using fixer.io by tonyplaysguitar. 
We'll use these fields, as this also have the "goal" in US-Dollars and we remove the other columns


In [ ]:
# remove unused columns, we don't need name and id, and the mentioned amounts
# we also do not need currency, as we have everything in US-Dollar
ks=ks_raw.drop(["ID", "name", "goal", "pledged", "usd pledged", "currency"], axis=1)

# we take a look at the datatypes, to look if we need to convert any fields to the appropriate data type
ks.dtypes 

We see that all the fields, that we need in a numeric form, are already automatically detected by python as numeric. 
However, to use the dates correctly, we need to convert launched and deadline into a datetime-datatype

In [ ]:
ks["launched"] = pd.to_datetime(ks["launched"], infer_datetime_format=True)
ks["deadline"] = pd.to_datetime(ks["deadline"], infer_datetime_format=True)

In [ ]:
# check for open projects
closed=ks["deadline"]>ks["launched"]
closed.value_counts()

We see that we only have closed projects here. As a prospect, we could include open projects and do a prediction on whether a project will be successful or not. Next step is to look at the data more closely and see some basic information on the data. For this we use the package pandas-profiling. See documenation [here](https://github.com/pandas-profiling/pandas-profiling)


In [ ]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(ks)

So we have a basic summary first. The data seems to be complete, there are no missing values, so we do not have to deal with that. However, if we look closely at **"launched"**, which is the launch-day of a product, we can see that there are some in 1970. These might be missing values, so we have to think about what to do with that. As we have plenty records (~380 000) and the number of those observations are relatively low, it is enough if we just remove those observations

Another thing is, that the cardinality of **"category"** is very high, meaning there are many different values. If in the end, the number of observations per category is too small, it might not make much sense to explore them further. We will elaborate on that later more closely.

The **countries** are only displayed with their country code. We need to convert them to proper country names for better readability, later on also possibly some geocoding, and thus using maps for visualizations and such things. We will also elaborate on that later.

Pandas_Profiling also provides a really quick Correlation Analysis. We can see a high positive correlation between backers und pledged amount, which makes perfect sense, so this does not give us particularly interesting information. In fact, it is more interesting to see the average amount that backers give for the projects. Thus, we'll look into this later.

In [ ]:
## remove the rows with launched in 1970
ks = ks[(ks["launched"].dt.year > 1970)]

In [ ]:
# read in a mapping file for countries
country_mapping=pd.read_csv("country.csv")

In [ ]:
country_mapping

In [ ]:
# we'll do a left outer join, with our Kickstarter-Projectlist being left
ks= pd.merge(ks,country_mapping, how="left", left_on="country", right_on="Code" )

# we can see that the names are now properly written out
# but we'll need to remove the joining columns and rename the Country column from "Name" to "Country"
ks= ks.drop(["country", "Code"], axis=1)
ks=ks.rename(columns={"Name": "Country"})
ks.tail()

## Distributions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#### First we take a look at the distribution of the status of the Projects


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
sns.countplot(x="state",data=ks, ax=ax,palette="Paired")


#### We can see that most Projects have failed. The values "live", "undefined" and "suspended" have the least values and their status is not clear. So these values will be removed in the next step. We concentrate on the more meaningful values "failed", "canceled" and "successful"

#### Removing Rows

In [ ]:
ks= ks[ks.state != "live"]
ks= ks[ks.state != "undefined"]
ks= ks[ks.state != "suspended"]

#### Distribution of the Main Category

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="main_category",data=ks, ax=ax, order = ks['main_category'].value_counts().index)

#### We have a deeper look at the distribution of categories and main categories

In [ ]:
ks.groupby("main_category")["category"].value_counts()


#### How is the distribution of failed,canceld or successful Projects in each category?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="main_category",hue="state",data=ks, ax=ax, palette="Paired")

#### How many entries in the dataset are from which country?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="country",data=ks,order = ks['country'].value_counts().index,ax=ax)

#### Distribution of the Project state across the countries

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.countplot(x="country",hue="state",data=ks, ax=ax, palette="Paired")

#### How many backers do the projects have in each country?

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.barplot(x="country",y="backers",data=ks,ci=None, ax=ax)
plt.show()

In [ ]:
print("Minimal goal value:", ks["usd_goal_real"].min())
print("Mean goal value:", ks["usd_goal_real"].mean())
print("Maximum goal value:", ks["usd_goal_real"].max())